# AstrbotTasks
- Taskiq再封装测试

首先要特别注意：
    InmemoryBroker比broker标准多几个方法，并且不可监听！


In [68]:
from taskiq import InMemoryBroker

# 基本任务分发代理
broker = InMemoryBroker()

# InmemoryResultBackend
- 作用：用于 InMemoryBroker 的结果存储，所有结果都保存在内存的有序字典中。
- 最大存储数：默认最多保存 100 条结果（可通过 max_stored_results 参数调整，-1 表示无限制）。
## 主要属性：
- results：保存任务结果的 OrderedDict，key 为 task_id。
- progress：保存任务进度的 OrderedDict，key 为 task_id。
## 核心方法：
- set_result(task_id, result)：保存任务结果，超出最大数量时自动移除最早的结果。
- is_result_ready(task_id)：检查指定任务结果是否已就绪。
- get_result(task_id)：获取指定任务的结果，不存在则抛异常。
- set_progress(task_id, progress)：保存任务进度，超出最大数量时自动移除最早的进度。
- get_progress(task_id)：获取指定任务的进度，若无则返回 None。
# InMemoryBroker
## 作用：本地开发用的内存型任务分发代理，不依赖外部消息队列。
## 主要参数：
- sync_tasks_pool_size：同步任务线程池大小，默认 4。
- max_stored_results：最大结果存储数，默认 100。
- max_async_tasks：最大异步任务数，默认 30。
- await_inplace：是否在 kick 时直接 await 任务。
## 核心属性：
- result_backend：结果后端，使用 InmemoryResultBackend。
- executor：线程池执行器。
- receiver：任务接收器，负责参数校验和任务调度。
- _running_tasks：当前正在运行的异步任务集合。
## 核心方法：
- kick(message)：执行指定任务，支持 await_inplace 或异步调度。
- listen()：不支持监听，调用会抛出 RuntimeError。<span style="color:red">(特别注意！其他broker都行！)</span>
- wait_all()：等待所有正在运行的任务完成，常用于测试场景。<span style="color:red">（特！别！注！意！其他！broker!没!）</span>
- startup() / shutdown()：分别触发启动和关闭事件，管理生命周期。

In [69]:
# 定义任务


@broker.task("sample_task", a="label_a", b="label_b")
async def sample_task(x: int, y: int) -> int:
    return x + y

# 下面假设在另一个文件发送（不！可！以！直接导入任务！）
- 可以通过事件调度（内置事件）
- 可以通过任务名调度

In [70]:
from typing import Any
from taskiq import (
    AsyncTaskiqDecoratedTask,
    AsyncTaskiqTask,
    TaskiqEvents,
    TaskiqResult,
    TaskiqState,
)


@broker.on_event(TaskiqEvents.CLIENT_STARTUP)
async def on_startup(state: TaskiqState):
    print("Broker is starting up...")


@broker.on_event(TaskiqEvents.CLIENT_STARTUP)
async def on_client_startup(state: TaskiqState):
    print("Client is starting up 2...")


@broker.on_event(TaskiqEvents.WORKER_SHUTDOWN)
async def on_shutdown(state: TaskiqState):
    print("Broker is shutting down...")


await broker.startup()

await broker.shutdown()

st: AsyncTaskiqDecoratedTask[Any, Any] | None = broker.find_task("sample_task")
if st is None:
    raise ValueError("Task not found")
await st.kiq(x=1, y=2)
task: AsyncTaskiqTask[Any] = await st.kiq(x=1, y=2)
result: TaskiqResult[Any] = await task.wait_result()

print(f"Task result: {result}")  # 输出: Task result:
print(f"Task result: {result.return_value}")  # 输出: 3

Broker is starting up...
Client is starting up 2...
Broker is shutting down...
Task result: is_err=False log=None return_value=3 execution_time=0.0 labels={'a': 'label_a', 'b': 'label_b'} error=None
Task result: 3
Task result: is_err=False log=None return_value=3 execution_time=0.0 labels={'a': 'label_a', 'b': 'label_b'} error=None
Task result: 3


In [71]:
# 同名任务覆盖（局部覆盖全局就不需要测了，AsyncBroker源代码可以看到）


@broker.task("sample_task", a="label_a", b="label_b")
async def sample_task(x: int, y: int) -> int:
    return x * y


st2: AsyncTaskiqDecoratedTask[Any, Any] | None = broker.find_task("sample_task")
if st2 is None:
    raise ValueError("Task not found")

task2: AsyncTaskiqTask[Any] = await st2.kiq(x=3, y=4)
result2: TaskiqResult[Any] = await task2.wait_result()
print(f"Task result after override: {result2.return_value}")  # 输出: 12

Task result after override: 12


In [72]:
# 自定义scheme

from yarl import URL

url = URL("astrbot://echo?foo=bar&baz=qux")
print(url.scheme)  # astrbot
print(url.host)  # echo
print(url.path)  # /
print(url.query)  # <MultiDictProxy()>

astrbot
echo
/
<MultiDictProxy('foo': 'bar', 'baz': 'qux')>


In [74]:
# 任务：


@broker.task("astrbot://core/echo")
async def echo_task(message: str) -> str:
    return message


@broker.task("astrbot://core/info")
async def info_task() -> str:
    return "This is AstrBot Canary"


# 封装


class AstrbotTaskService:
    broker = broker

    @classmethod
    async def get(cls, url: URL):
        task_name = f"{url.scheme}://{url.host}{url.path}"
        task_handler: AsyncTaskiqDecoratedTask[Any, str] | None = cls.broker.find_task(
            task_name
        )
        query = dict(url.query)
        if task_handler is None:
            raise ValueError(f"Task {task_name} not found")
        task = await task_handler.kiq(**query)
        result = await task.wait_result()
        return result.return_value

    @classmethod
    async def post(cls, url: URL, data: dict[str, Any]):
        task_name = f"{url.scheme}://{url.host}{url.path}"
        task_handler: AsyncTaskiqDecoratedTask[Any, str] | None = cls.broker.find_task(
            task_name
        )
        if task_handler is None:
            raise ValueError(f"Task {task_name} not found")
        task = await task_handler.kiq(**data)
        result = await task.wait_result()
        return result.return_value

    @classmethod
    async def put(cls, url: URL, data: dict[str, Any]):
        task_name = f"{url.scheme}://{url.host}{url.path}"
        task_handler: AsyncTaskiqDecoratedTask[Any, str] | None = cls.broker.find_task(
            task_name
        )
        if task_handler is None:
            raise ValueError(f"Task {task_name} not found")
        # put方法可以和post一样，实际根据业务区分
        task = await task_handler.kiq(**data)
        result = await task.wait_result()
        return result.return_value

    @classmethod
    async def delete(cls, url: URL, data: dict[str, Any] = None):
        task_name = f"{url.scheme}://{url.host}{url.path}"
        task_handler: AsyncTaskiqDecoratedTask[Any, str] | None = cls.broker.find_task(
            task_name
        )
        if task_handler is None:
            raise ValueError(f"Task {task_name} not found")
        # delete方法通常只需要url，但如有参数可传data
        kwargs = data if data else {}
        task = await task_handler.kiq(**kwargs)
        result = await task.wait_result()
        return result.return_value

    @classmethod
    async def patch(cls, url: URL, data: dict[str, Any]):
        task_name = f"{url.scheme}://{url.host}{url.path}"
        task_handler: AsyncTaskiqDecoratedTask[Any, str] | None = cls.broker.find_task(
            task_name
        )
        if task_handler is None:
            raise ValueError(f"Task {task_name} not found")
        # patch方法用于部分更新
        task = await task_handler.kiq(**data)
        result = await task.wait_result()
        return result.return_value

    @classmethod
    async def head(cls, url: URL):
        task_name = f"{url.scheme}://{url.host}{url.path}"
        task_handler: AsyncTaskiqDecoratedTask[Any, str] | None = cls.broker.find_task(
            task_name
        )
        if task_handler is None:
            raise ValueError(f"Task {task_name} not found")
        # head方法只关心元信息
        return {
            "task_name": task_name,
            "task_labels": task_handler.labels,
            "broker": str(type(cls.broker)),
            "backend": str(type(cls.broker.result_backend)),
        }

    @classmethod
    async def options(cls, url: URL):
        task_name = f"{url.scheme}://{url.host}{url.path}"
        # options方法通常返回支持的方法，这里简单模拟
        return {"allow": ["GET", "POST", "PUT", "DELETE", "PATCH", "HEAD", "OPTIONS"]}


r1 = await AstrbotTaskService.get(URL("astrbot://core/echo?message=Hello%20World"))

r2 = await AstrbotTaskService.post(
    URL("astrbot://core/echo"), data={"message": "Hello World2"}
)
r3 = await AstrbotTaskService.put(
    URL("astrbot://core/echo"), data={"message": "Hello PUT"}
)
r4 = await AstrbotTaskService.delete(
    URL("astrbot://core/echo"), data={"message": "Hello DELETE"}
)
r5 = await AstrbotTaskService.patch(
    URL("astrbot://core/echo"), data={"message": "Hello PATCH"}
)
r6 = await AstrbotTaskService.head(URL("astrbot://core/echo"))
r7 = await AstrbotTaskService.options(URL("astrbot://core/echo"))
print(r1)
print(r2)
print(r3)
print(r4)
print(r5)
print(r6)
print(r7)

Hello World
Hello World2
Hello PUT
Hello DELETE
Hello PATCH
{'task_name': 'astrbot://core/echo', 'task_labels': {}, 'broker': "<class 'taskiq.brokers.inmemory_broker.InMemoryBroker'>", 'backend': "<class 'taskiq.brokers.inmemory_broker.InmemoryResultBackend'>"}
{'allow': ['GET', 'POST', 'PUT', 'DELETE', 'PATCH', 'HEAD', 'OPTIONS']}


In [80]:
# FastAPI风格注册 + requests风格请求响应
from typing import Any, Callable, Awaitable, Dict, Optional, TypeVar, Generic
from yarl import URL
from taskiq import InMemoryBroker, AsyncTaskiqDecoratedTask

T = TypeVar("T")
RequestType = Dict[str, Any]


class Response(Generic[T]):
    def __init__(
        self, data: T, status_code: int = 200, headers: Optional[Dict[str, Any]] = None
    ):
        self.data = data
        self.status_code = status_code
        self.headers = headers or {}

    def json(self) -> T:
        return self.data


class App:
    def __init__(self):
        self.broker = InMemoryBroker()
        self.routes: Dict[
            tuple[str, str], Callable[[RequestType], Awaitable[Response[Any]]]
        ] = {}
        self.requests = self.Requests(self)

    def get(
        self, path: str
    ) -> Callable[
        [Callable[[RequestType], Awaitable[Any]]],
        Callable[[RequestType], Awaitable[Any]],
    ]:
        def decorator(
            func: Callable[[RequestType], Awaitable[Any]],
        ) -> Callable[[RequestType], Awaitable[Any]]:
            task_name = f"astrbot://core{path}"

            @self.broker.task(task_name)
            async def wrapper(**kwargs):
                result = await func(kwargs)
                return Response(result)

            self.routes[("GET", path)] = func
            return func

        return decorator

    def post(
        self, path: str
    ) -> Callable[
        [Callable[[RequestType], Awaitable[Any]]],
        Callable[[RequestType], Awaitable[Any]],
    ]:
        def decorator(
            func: Callable[[RequestType], Awaitable[Any]],
        ) -> Callable[[RequestType], Awaitable[Any]]:
            task_name = f"astrbot://core{path}"

            @self.broker.task(task_name)
            async def wrapper(**kwargs):
                result = await func(kwargs)
                return Response(result)

            self.routes[("POST", path)] = func
            return func

        return decorator

    # 可扩展put/delete/patch...
    class Requests:
        def __init__(self, app: "App"):
            self.app = app

        async def get(
            self, url: str, params: Optional[RequestType] = None
        ) -> Response[Any]:
            return await self._call("GET", url, params or {})

        async def post(
            self, url: str, data: Optional[RequestType] = None
        ) -> Response[Any]:
            return await self._call("POST", url, data or {})

        async def _call(
            self, method: str, url: str, data: RequestType
        ) -> Response[Any]:
            url_obj = URL(url)
            task_name = f"{url_obj.scheme}://{url_obj.host}{url_obj.path}"
            task_handler: Optional[AsyncTaskiqDecoratedTask[Any, Any]] = (
                self.app.broker.find_task(task_name)
            )
            if not task_handler:
                raise ValueError(f"Task {task_name} not found")
            task = await task_handler.kiq(**data)
            result = await task.wait_result()
            # 兼容Response包装
            if isinstance(result.return_value, Response):
                return result.return_value
            return Response(result.return_value)


app = App()


@app.get("/echo")
async def echo(request: RequestType) -> str:
    return f"[Router] {request.get('message', '')}"


@app.post("/add")
async def add(request: RequestType) -> int:
    return int(request.get("a", 0)) + int(request.get("b", 0))


# 用例：requests风格调用，返回Response对象
resp1 = await app.requests.get(
    "astrbot://core/echo", params={"message": "Hello Router"}
)
resp2 = await app.requests.post("astrbot://core/add", data={"a": 1, "b": 2})
print(resp1.data)  # 输出: [Router] Hello Router
print(resp2.data)  # 输出: 3

[Router] Hello Router
3
